In [1]:
import sys
import os
import string
import logging
import glob

from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import datetime

import pyresample as pr
import math, cmath

ModuleNotFoundError: No module named 'netCDF4'

In [ ]:
def iskoncentration(aar,maaned,dag):
    #resampling data
    #
    
    #filename1 er fornavnet på filen: /home/rtt/code/ESMR/filtered
    #/media/rtt/Seagate Expansion Drive/data/N5ESMR/processed_v1
    ncfile_n="Nimbus5-ESMR_"+aar+"m"+maaned+dag+"ease_n.nc"
    #ncfile_path_n=os.path.join('/','home','rtt','data','ESMR','test_data',ncfile_n)
    ncfile_path_n=os.path.join('/','media','rtt','Seagate Expansion Drive','data','N5ESMR',\
                               'processed_v1',ncfile_n)
    
    ncfile_s="Nimbus5-ESMR_"+aar+"m"+maaned+dag+"ease_s.nc"
    #ncfile_path_s=os.path.join('/','home','rtt','data','ESMR','test_data',ncfile_s)
    ncfile_path_s=os.path.join('/','media','rtt','Seagate Expansion Drive','data','N5ESMR',\
                               'processed_v1',ncfile_s)

    lats_list = []
    lons_list = []
    sst_list = []
    t2m_list = []
    tcw_list = []
    tcwv_list = []
    u10_list = []
    v10_list = []
    siconc_list = []
    brightness_temperature_list = []

    #producerer filename_list:: change path name here...
    #fx.: /media/rtt/Seagate Expansion Drive/data/N5ESMR/ESMR_1974_filtered
    folder_name="ESMR_"+aar+"_filtered"
    filename2="Nimbus5-ESMR_"+aar+"m"+maaned+dag+"*.nc"
    print(filename2)
    #dag i fx. januar 1974
    afile=os.path.join('/','media','rtt','Seagate Expansion Drive','data','N5ESMR',folder_name,filename2)
    print(afile)

    filename_list = glob.glob(afile)
    for filename in filename_list:
        print(filename)
        ds = Dataset(filename, 'r')
        lats_list.append(ds.variables['Latitude'][:])
        lons_list.append(ds.variables['Longitude'][:])
        sst_list.append(ds.variables['sst'][:])
        t2m_list.append(ds.variables['t2m'][:])
        tcw_list.append(ds.variables['tcw'][:])
        tcwv_list.append(ds.variables['tcwv'][:])
        u10_list.append(ds.variables['u10'][:])
        v10_list.append(ds.variables['v10'][:])
        siconc_list.append(ds.variables['siconc'][:])
        brightness_temperature_list.append(ds.variables['Brightness_temperature'][:])
        ds.close()

    lat =   np.concatenate(lats_list)
    lon =   np.concatenate(lons_list)
    
    sst_e40 = np.concatenate(sst_list)
    t2m_e40 = np.concatenate(t2m_list)
    tcw_e40 = np.concatenate(tcw_list)
    tcwv_e40 = np.concatenate(tcwv_list)
    u10_e40 = np.concatenate(u10_list)
    v10_e40 = np.concatenate(v10_list)
    sic_e40 = np.concatenate(siconc_list)
    Tb = np.concatenate(brightness_temperature_list)
    
    #just use the midswath here [:][4:73]:: this is for ESMR and it is not needed if the whole swath should be used
    mid_swath = [ 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,\
                 24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,\
                 40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,\
                 57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73]
    
    lat =   lat[:,mid_swath]
    lon =   lon[:,mid_swath]
    sst_e40 = sst_e40[:,mid_swath]
    t2m_e40 = t2m_e40[:,mid_swath]
    tcw_e40 = tcw_e40[:,mid_swath]
    tcwv_e40 = tcwv_e40[:,mid_swath]
    u10_e40 = u10_e40[:,mid_swath]
    v10_e40 = v10_e40[:,mid_swath]
    sic_e40 = sic_e40[:,mid_swath]
    Tb = Tb[:,mid_swath]
    
    clw = tcw_e40 - tcwv_e40
    wind=np.sqrt(u10_e40**2+v10_e40**2)
    
    
    #Just look at relevant areas
    ocean=(sic_e40 == 0)
    warm_water = (sst_e40 > 278.0)
    ocean_mask = ocean & warm_water
    print("number of ocean points masked away",np.sum(np.sum(ocean_mask)))
    #is er kun i koldt vand, hvis vandet er 5 grader varmt er der nok ikke is i det.
    ice_infested_waters = (sst_e40 <= 278.0)
    open_water = (sst_e40 > 278.0)
    #geografisk udvaelgelse, her var det før lat > 0 og lat < 0, der er ingen is syd for 78.40 deg.
    arctic = (lat > 32)
    north_pole=(lat < 90)
    antarctic =(lat < -48)
    south_pole=(lat > -90)
    #arctic_ice = arctic & ice_infested_waters
    arctic_ice = arctic & north_pole
    arctic_open_water = arctic & open_water
    #antarctic_ice = antarctic & ice_infested_waters
    antarctic_ice = antarctic & south_pole
    antarctic_open_water = antarctic & open_water
    print("number of points in ice infested waters used for computation",\
          np.sum(np.sum(ice_infested_waters)))
    
    #definer koncentrations arrays, not working
    arctic_sic=np.zeros(Tb[arctic_ice+arctic_open_water].shape)
    antarctic_sic=np.zeros(Tb[antarctic].shape)
    
    #stralingstemperaturer: is
    Tb_ice_arctic=Tb[arctic_ice]
    Tb_ice_antarctic=Tb[antarctic_ice]
    print("Tb_ice_arctic.shape",Tb_ice_arctic.shape)
    
    #compute the sea ice concentration here... and the ice type...
    ###############################################################
    
    #resample the grids
    lon_n, lat_n = pr.utils.check_and_wrap(lon[arctic_ice], lat[arctic_ice])
    lon_s, lat_s = pr.utils.check_and_wrap(lon[antarctic_ice], lat[antarctic_ice])

    #swath_def = pr.geometry.SwathDefinition(lons=lon, lats=lat)
    
    arctic_def = pr.geometry.SwathDefinition(lons=lon_n, lats=lat_n)
    antarctic_def = pr.geometry.SwathDefinition(lons=lon_s, lats=lat_s)

    #now we do the mapping of the swathdata
    #now get the grid definition from the cfg file
    area_def_north_ease = pr.utils.parse_area_file('./areas_ease.cfg', 'ease_nh')[0]
    area_def_south_ease = pr.utils.parse_area_file('./areas_ease.cfg', 'ease_sh')[0]

    #resample
    
    n_Tb = pr.kd_tree.resample_nearest(arctic_def, Tb_ice_arctic,\
               area_def_north_ease, radius_of_influence=50000,fill_value=None)

    s_Tb = pr.kd_tree.resample_nearest(antarctic_def, Tb_ice_antarctic,\
               area_def_south_ease, radius_of_influence=50000,fill_value=None)

   
    #landmask
    dn = Dataset('lmask_nh_ease_250.nc', 'r')
    landnn = dn.variables['lmask'][:]
    dn.close
    
    land_mask_arctic=np.copy(landnn)
    land_mask_arctic[land_mask_arctic > 0.5]=1
    #the datapoint is over land, flag 1
    
    ds = Dataset('lmask_sh_ease_250.nc', 'r')
    landss = ds.variables['lmask'][:]
    ds.close
    
    land_mask_antarctic=np.copy(landss)
    
    plt.imshow(n_Tb,vmin=100,vmax=300)
    plt.show()
    
    plt.imshow(s_Tb,vmin=100,vmax=300)
    plt.show()
    
    filename1=str(aar+"m"+maaned+dag)
    
    pngfile_n_tb="N5_ESMR_Tb"+filename1+"_n_qlook.png"
    pngfile_n_path_tb=os.path.join('/','home','rtt','data','ESMR','test_data',pngfile_n_tb)
    
    pngfile_s_tb="N5_ESMR_Tb"+filename1+"_s_qlook.png"
    pngfile_s_path_tb=os.path.join('/','home','rtt','data','ESMR','test_data',pngfile_s_tb)

    #plot quicklooks using pyresample
    pr.plot.save_quicklook(pngfile_n_path_tb, area_def_north_ease, n_Tb, \
                           label='brightness temperature [K]',vmin=100.0,vmax=300.0,cmap='coolwarm')

    pr.plot.save_quicklook(pngfile_s_path_tb, area_def_south_ease, s_Tb, \
                           label='brightness temperature [K]',vmin=100.0,vmax=300.0,cmap='coolwarm')

In [ ]:
iskoncentration('1972','12','28')